In [46]:
# !pip install torchinfo
# !pip install torcheval

In [47]:
# from google.colab import drive
# drive.mount('/content/drive')


In [48]:
# %cd /content/drive/MyDrive/2023_AI_new

In [1]:
import datetime
import os
import time
from tqdm import tqdm
import torch
import h5py
import numpy as np
from torchinfo import summary
import math
from torch.autograd import Variable
import pickle

In [2]:
# custom module
import model, pamap_dataloader, train_model, test_model
import importlib
importlib.reload(model)
importlib.reload(pamap_dataloader)
importlib.reload(train_model)
importlib.reload(test_model)
from model import *
from data import *
from pamap_dataloader import *
from train_model import train
from test_model import test

# Parameters

In [3]:
config = {
    "root" : "./data/pickles/",
    "train_feature" : "PAMAP2_feature.pickle",
    "train_target" : "PAMAP2_target.pickle",
    "test_feature" : "PAMAP2_feature_test.pickle",
    "test_target" : "PAMAP2_target_test.pickle",
    "labelencodedict" :  { 1 : 0,
                   3 : 1,
                   4 : 2,
                   6 : 3,
                   7 : 4,
                   16 : 5,
                   17 : 6
                   },
    "epochs" : 3000,
    "learning_rate" : 1e-4,
    # model 
    "first_out_channels" : 32,
    "first_kernel_size" : 5,
    "second_in_channels" : 32,
    "second_out_channels" : 16,
    "second_kernel_size" : 3,
    "dim_model" : 128,
    "dim_inner" : 256,
    "num_heads" : 8,
    "dropout_rate" : 0.15,
    "squeeze_factor" : 2,
    "sec_kernel_size" : 3,
    "sec_stride" : 1,
    "N" : 3,
    "gamma" : 2,
    "u" : 2,     
    "weight_decay" : 0.1,
    "device" : torch.device('cuda' if torch.cuda.is_available() else 'cpu')
}

# Main

In [ ]:
if __name__ == "__main__":
    
    epochs = config["epochs"]
    learning_rate = config["learning_rate"]
    weight_decay = config["weight_decay"]
    X_train, y_train, train_y, X_test, y_test, test_y = load_dataset(config)
    
    X_train = X_train.double()
    X_test = X_test.double()
    config["num_samples"] = X_train.shape[0]
    config["first_in_channels"] = X_train.shape[1]
    config["L"] = X_train.shape[2]
    config["length"] = X_train.shape[2]
    
    md = Net(
                dim_model = config["dim_model"],
                gamma = config["gamma"],
                u = config["u"],
                ys = y_train,
                first_in_channels = config["first_in_channels"],
                first_out_channels = config["first_out_channels"],
                first_kernel_size = config["first_kernel_size"],
                second_in_channels = config["second_in_channels"],
                second_out_channels = config["second_out_channels"],
                second_kernel_size = config["second_kernel_size"],
                N = config["N"],  #slice
                L = config["L"], # slice
                dim_inner = config["dim_inner"],
                num_samples = config["num_samples"], 
                length = config["length"], 
                num_heads = config["num_heads"],
                dropout_rate = config["dropout_rate"],
                squeeze_factor = config["squeeze_factor"],
                sec_kernel_size = config["sec_kernel_size"],
                sec_stride = config["sec_stride"],
                num_classes = len(config["labelencodedict"].keys())).to(config['device'])
    optimizer = torch.optim.Adam(md.parameters(),weight_decay=weight_decay, lr=learning_rate)
    criterion = torch.nn.CrossEntropyLoss()
    
    train_loss, train_acc, train_f1  = train(X_train, train_y, optimizer, model = md, epochs = epochs, f1=True)
    md.ys = y_test
    test_loss, test_acc, test_f1  = test(X_test, test_y, model = md, f1 = True)

C:\Users\yuins\Hanyang_23_1st\AI\model.py:40: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  attention_prob = torch.nn.functional.softmax(scaled_att_score)
C:\Users\yuins\Hanyang_23_1st\AI\model.py:255: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  attention_prob = torch.nn.functional.softmax(scaled_att_score)


Epoch: 0, Loss: 1.0578, Train acc: 97.5510%, F1 macro: 0.9140
Epoch: 1, Loss: 0.7992, Train acc: 99.1837%, F1 macro: 0.9711
Epoch: 2, Loss: 0.8081, Train acc: 98.3673%, F1 macro: 0.9444
Epoch: 3, Loss: 0.6659, Train acc: 97.5510%, F1 macro: 0.9163
Epoch: 4, Loss: 0.5327, Train acc: 95.9184%, F1 macro: 0.8632
Epoch: 5, Loss: 0.4364, Train acc: 97.5510%, F1 macro: 0.9116
Epoch: 6, Loss: 0.4370, Train acc: 99.1837%, F1 macro: 0.9711
Epoch: 7, Loss: 0.3500, Train acc: 99.1837%, F1 macro: 0.9711
Epoch: 8, Loss: 0.3132, Train acc: 98.3673%, F1 macro: 0.9423
Epoch: 9, Loss: 0.1957, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 10, Loss: 0.1520, Train acc: 99.1837%, F1 macro: 0.9711
Epoch: 11, Loss: 0.1441, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 12, Loss: 0.0738, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 13, Loss: 0.0755, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 14, Loss: 0.0794, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 15, Loss: 0.0423, Train acc: 100.0000%, F1 macro: 1.0

Epoch: 128, Loss: 0.0033, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 129, Loss: 0.0029, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 130, Loss: 0.0033, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 131, Loss: 0.0037, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 132, Loss: 0.0041, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 133, Loss: 0.0035, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 134, Loss: 0.0024, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 135, Loss: 0.0033, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 136, Loss: 0.0027, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 137, Loss: 0.0045, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 138, Loss: 0.0024, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 139, Loss: 0.0035, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 140, Loss: 0.0033, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 141, Loss: 0.0037, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 142, Loss: 0.0036, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 143, Loss: 0.0076,

Epoch: 255, Loss: 0.0021, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 256, Loss: 0.0035, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 257, Loss: 0.0022, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 258, Loss: 0.0025, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 259, Loss: 0.0022, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 260, Loss: 0.0035, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 261, Loss: 0.0020, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 262, Loss: 0.0022, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 263, Loss: 0.0024, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 264, Loss: 0.0033, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 265, Loss: 0.0027, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 266, Loss: 0.0023, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 267, Loss: 0.0035, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 268, Loss: 0.0043, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 269, Loss: 0.0050, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 270, Loss: 0.0176,

Epoch: 382, Loss: 0.0018, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 383, Loss: 0.0021, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 384, Loss: 0.0027, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 385, Loss: 0.0011, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 386, Loss: 0.0016, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 387, Loss: 0.0016, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 388, Loss: 0.0014, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 389, Loss: 0.0020, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 390, Loss: 0.0012, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 391, Loss: 0.0019, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 392, Loss: 0.0013, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 393, Loss: 0.0018, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 394, Loss: 0.0017, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 395, Loss: 0.0019, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 396, Loss: 0.0012, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 397, Loss: 0.0016,

Epoch: 509, Loss: 0.0003, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 510, Loss: 0.0004, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 511, Loss: 0.0005, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 512, Loss: 0.0005, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 513, Loss: 0.0006, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 514, Loss: 0.0004, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 515, Loss: 0.0004, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 516, Loss: 0.0004, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 517, Loss: 0.0005, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 518, Loss: 0.0004, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 519, Loss: 0.0005, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 520, Loss: 0.0005, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 521, Loss: 0.0006, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 522, Loss: 0.0004, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 523, Loss: 0.0007, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 524, Loss: 0.0007,

Epoch: 636, Loss: 0.0008, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 637, Loss: 0.0011, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 638, Loss: 0.0008, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 639, Loss: 0.0009, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 640, Loss: 0.0008, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 641, Loss: 0.0011, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 642, Loss: 0.0011, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 643, Loss: 0.0012, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 644, Loss: 0.0014, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 645, Loss: 0.0009, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 646, Loss: 0.0009, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 647, Loss: 0.0008, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 648, Loss: 0.0008, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 649, Loss: 0.0009, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 650, Loss: 0.0014, Train acc: 100.0000%, F1 macro: 1.0000
Epoch: 651, Loss: 0.0009,

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (10,6))
plt.ylim([-0.1, 1.5])
plt.title("Train Loss", fontsize = 20)
plt.xlabel("Epochs", fontsize=15)
plt.ylabel("Loss", fontsize=15)
plt.axhline(1, alpha = 0.1, color = "black", linestyle = "--")
plt.plot([i.cpu().detach().numpy() for i in train_loss])


plt.figure(figsize = (10,6))
plt.ylim([90, 102])
plt.title("Train Accuracy", fontsize = 20)
plt.xlabel("Epochs", fontsize=15)
plt.ylabel("Accuracy", fontsize=15)
plt.axhline(100, alpha = 0.1, color = "black", linestyle = "--")
plt.plot([i.cpu().detach().numpy() for i in train_acc])


In [ ]:
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import seaborn as sns

model_dimreduce = TSNE(n_components=2, learning_rate='auto',
                  init='pca', perplexity=3)
scaler = StandardScaler()

for i in range(len(train_representation)):
    _rep = train_representation[i].cpu().detach().numpy()
    _class = train_prototypes[i].squeeze(dim =1).cpu().detach().numpy()

    _rep_tsne = model_dimreduce.fit_transform(_rep)
    _class_tsne = model_dimreduce.fit_transform(_class)

    _rep_tsne = scaler.fit_transform(_rep)
    _class_tsne = scaler.transform(_class)
    plt.title("Epoch {0}".format((i * 250)), fontsize=20)
    sns.scatterplot(x =_rep_tsne[:, 0], y = _rep_tsne[:, 1], label = "Representation")
    sns.scatterplot(x =_class_tsne[:, 0], y = _class_tsne[:, 1], marker = "*", label = "Class prototypes", s=100)
    plt.legend()
    plt.show()